In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
from dataclasses import dataclass
from typing import List, Tuple, Optional
import copy

# Board constants
WHITE, BLACK = 'w', 'b'
PIECES = ['K', 'Q', 'R', 'B', 'N', 'P']
UNICODE = {
    (WHITE, 'K'): '♔', (WHITE, 'Q'): '♕', (WHITE, 'R'): '♖', (WHITE, 'B'): '♗', (WHITE, 'N'): '♘', (WHITE, 'P'): '♙',
    (BLACK, 'K'): '♚', (BLACK, 'Q'): '♛', (BLACK, 'R'): '♜', (BLACK, 'B'): '♝', (BLACK, 'N'): '♞', (BLACK, 'P'): '♟',
}

SQ = 80 # square size
MARGIN = 20
BOARD_SIZE = SQ * 8
FONT = ("DejaVu Sans", 42)
LIGHT = '#f0d9b5'
DARK = '#b58863'
HILITE = '#f6f669'
MOVE_HILITE = '#a2d5f2'
CHECK_HILITE = '#ff6b6b'

@dataclass
class Move:
    src: Tuple[int, int]
    dst: Tuple[int, int]
    promotion: Optional[str] = None # one of 'Q','R','B','N'


def initial_board():
    # 8x8 board indexed [r][c]; r=0 at top (Black's back rank), r=7 at bottom (White's back rank)
    B = [[None for _ in range(8)] for _ in range(8)]
    back = ['R','N','B','Q','K','B','N','R']
    for c, p in enumerate(back):
        B[0][c] = (BLACK, p)
        B[7][c] = (WHITE, p)
    for c in range(8):
        B[1][c] = (BLACK, 'P')
        B[6][c] = (WHITE, 'P')
    return B


def in_bounds(r, c):
    return 0 <= r < 8 and 0 <= c < 8


def locate_king(board, color):
    for r in range(8):
        for c in range(8):
            if board[r][c] == (color, 'K'):
                return (r, c)
    return None


def gen_pseudo_moves(board, color):
    moves = []
    for r in range(8):
        for c in range(8):
            piece = board[r][c]
            if not piece or piece[0] != color:
                continue
            kind = piece[1]
            if kind == 'P':
                dir_ = -1 if color == WHITE else 1
                start_row = 6 if color == WHITE else 1
                # one step forward
                nr = r + dir_
                if in_bounds(nr, c) and board[nr][c] is None:
                    # promotion
                    if nr == 0 or nr == 7:
                        for promo in ['Q','R','B','N']:
                            moves.append(Move((r,c),(nr,c), promo))
                    else:
                        moves.append(Move((r,c),(nr,c)))
                    # two steps from start
                    nnr = r + 2*dir_
                    if r == start_row and board[nnr][c] is None:
                        moves.append(Move((r,c),(nnr,c)))
                # captures
                for dc in (-1, 1):
                    nc = c + dc
                    nr = r + dir_
                    if in_bounds(nr, nc) and board[nr][nc] and board[nr][nc][0] != color:
                        if nr == 0 or nr == 7:
                            for promo in ['Q','R','B','N']:
                                moves.append(Move((r,c),(nr,nc), promo))
                        else:
                            moves.append(Move((r,c),(nr,nc)))
                # (en passant omitted)
            elif kind == 'N':
                for dr, dc in [(2,1),(2,-1),(-2,1),(-2,-1),(1,2),(1,-2),(-1,2),(-1,-2)]:
                    nr, nc = r+dr, c+dc
                    if in_bounds(nr, nc) and (board[nr][nc] is None or board[nr][nc][0] != color):
                        moves.append(Move((r,c),(nr,nc)))
            elif kind in ('B','R','Q'):
                dirs = []
                if kind in ('B','Q'):
                    dirs += [(-1,-1),(-1,1),(1,-1),(1,1)]
                if kind in ('R','Q'):
                    dirs += [(-1,0),(1,0),(0,-1),(0,1)]
                for dr, dc in dirs:
                    nr, nc = r+dr, c+dc
                    while in_bounds(nr, nc):
                        if board[nr][nc] is None:
                            moves.append(Move((r,c),(nr,nc)))
                        else:
                            if board[nr][nc][0] != color:
                                moves.append(Move((r,c),(nr,nc)))
                            break
                        nr += dr; nc += dc
            elif kind == 'K':
                for dr in (-1,0,1):
                    for dc in (-1,0,1):
                        if dr==0 and dc==0: continue
                        nr, nc = r+dr, c+dc
                        if in_bounds(nr, nc) and (board[nr][nc] is None or board[nr][nc][0] != color):
                            moves.append(Move((r,c),(nr,nc)))
                # (castling omitted)
    return moves


def square_attacked_by(board, sq, attacker_color):
    # Determine if sq=(r,c) is attacked by any piece of attacker_color
    r, c = sq
    # Knights
    for dr, dc in [(2,1),(2,-1),(-2,1),(-2,-1),(1,2),(1,-2),(-1,2),(-1,-2)]:
        nr, nc = r+dr, c+dc
        if in_bounds(nr,nc) and board[nr][nc] == (attacker_color, 'N'):
            return True
    # Kings (adjacent)
    for dr in (-1,0,1):
        for dc in (-1,0,1):
            if dr==0 and dc==0: continue
            nr, nc = r+dr, c+dc
            if in_bounds(nr,nc) and board[nr][nc] == (attacker_color, 'K'):
                return True
    # Sliding pieces
    def ray(dirs, attackers):
        for dr, dc in dirs:
            nr, nc = r+dr, c+dc
            while in_bounds(nr,nc):
                p = board[nr][nc]
                if p is None:
                    nr += dr; nc += dc; continue
                if p[0] == attacker_color and p[1] in attackers:
                    return True
                break
        return False
    if ray([(-1,0),(1,0),(0,-1),(0,1)], attackers={'R','Q'}):
        return True
    if ray([(-1,-1),(-1,1),(1,-1),(1,1)], attackers={'B','Q'}):
        return True
    # Pawns
    dir_ = -1 if attacker_color == WHITE else 1
    for dc in (-1,1):
        nr, nc = r - dir_, c - dc # invert since we're looking from target square
        if in_bounds(nr,nc) and board[nr][nc] == (attacker_color, 'P'):
            return True
    return False


def make_move(board, move: Move):
    b = copy.deepcopy(board)
    (r1,c1), (r2,c2) = move.src, move.dst
    piece = b[r1][c1]
    b[r1][c1] = None
    # promotion handling
    if piece[1] == 'P' and (r2 == 0 or r2 == 7) and move.promotion:
        b[r2][c2] = (piece[0], move.promotion)
    else:
        b[r2][c2] = piece
    return b


def in_check(board, color):
    kpos = locate_king(board, color)
    if not kpos:
        return True # king missing -> treat as check
    opp = WHITE if color == BLACK else BLACK
    return square_attacked_by(board, kpos, opp)


def legal_moves(board, color):
    legal = []
    for m in gen_pseudo_moves(board, color):
        newb = make_move(board, m)
        if not in_check(newb, color):
            legal.append(m)
    return legal


class ChessApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Chess - Tkinter")
        self.resizable(False, False)

        self.canvas = tk.Canvas(self, width=BOARD_SIZE, height=BOARD_SIZE)
        self.canvas.grid(row=0, column=0, columnspan=3)

        self.turn_label = tk.Label(self, text="White to move", font=("Helvetica", 14, "bold"))
        self.turn_label.grid(row=1, column=0, sticky='w', padx=10, pady=6)

        self.status_label = tk.Label(self, text="", fg="gray")
        self.status_label.grid(row=1, column=1, pady=6)

        self.reset_btn = tk.Button(self, text="Restart", command=self.reset)
        self.reset_btn.grid(row=1, column=2, sticky='e', padx=10)

        self.history = tk.Listbox(self, height=10, width=20)
        self.history.grid(row=0, column=3, padx=(10,10), pady=10, sticky='ns')
        tk.Label(self, text="Moves").grid(row=1, column=3)

        self.bind_events()
        self.reset()

    def reset(self):
        self.board = initial_board()
        self.to_move = WHITE
        self.selected = None
        self.legal_cache = legal_moves(self.board, self.to_move)
        self.history.delete(0, tk.END)
        self.draw()
        self.update_status()

    def bind_events(self):
        self.canvas.bind('<Button-1>', self.on_click)

    def rc_from_xy(self, x, y):
        return y // SQ, x // SQ

    def draw(self):
        self.canvas.delete('all')
        # Board squares
        for r in range(8):
            for c in range(8):
                x0, y0 = c*SQ, r*SQ
                color = LIGHT if (r+c)%2==0 else DARK
                self.canvas.create_rectangle(x0, y0, x0+SQ, y0+SQ, fill=color, outline=color)
        # Highlights
        if self.selected:
            r, c = self.selected
            self.canvas.create_rectangle(c*SQ, r*SQ, c*SQ+SQ, r*SQ+SQ, outline=HILITE, width=4)
            for m in self.legal_cache:
                if m.src == self.selected:
                    rr, cc = m.dst
                    self.canvas.create_oval(cc*SQ+SQ//2-10, rr*SQ+SQ//2-10, cc*SQ+SQ//2+10, rr*SQ+SQ//2+10, fill=MOVE_HILITE, outline='')
        # Check highlight
        if in_check(self.board, self.to_move):
            kpos = locate_king(self.board, self.to_move)
            if kpos:
                r, c = kpos
                self.canvas.create_rectangle(c*SQ, r*SQ, c*SQ+SQ, r*SQ+SQ, outline=CHECK_HILITE, width=4)
        # Pieces
        for r in range(8):
            for c in range(8):
                p = self.board[r][c]
                if p:
                    sym = UNICODE[p]
                    self.canvas.create_text(c*SQ+SQ//2, r*SQ+SQ//2, text=sym, font=FONT)

    def update_status(self):
        self.turn_label.config(text=("White to move" if self.to_move == WHITE else "Black to move"))
        moves = legal_moves(self.board, self.to_move)
        if not moves:
            if in_check(self.board, self.to_move):
                self.status_label.config(text=("Checkmate! " + ("Black" if self.to_move==WHITE else "White") + " wins."), fg='red')
            else:
                self.status_label.config(text="Stalemate.", fg='orange')
        else:
            self.status_label.config(text=f"Legal moves: {len(moves)}", fg='gray')
        self.legal_cache = moves

    def on_click(self, event):
        r, c = self.rc_from_xy(event.x, event.y)
        if not in_bounds(r,c):
            return
        p = self.board[r][c]
        if self.selected is None:
            if p and p[0] == self.to_move:
                self.selected = (r, c)
        else:
            src = self.selected
            # Is this a legal destination from selected?
            cand = [m for m in self.legal_cache if m.src == src and m.dst == (r,c)]
            if cand:
                m = cand[0]
                # handle promotion if needed
                piece = self.board[src[0]][src[1]]
                if piece[1] == 'P' and (r == 0 or r == 7):
                    choice = self.ask_promotion()
                    m = Move(src, (r,c), promotion=choice)
                self.board = make_move(self.board, m)
                self.push_history(m, piece)
                self.to_move = WHITE if self.to_move == BLACK else BLACK
                self.selected = None
                self.draw()
                self.update_status()
                return
            # else: select new square if same-color piece, or clear
            if p and p[0] == self.to_move:
                self.selected = (r, c)
            else:
                self.selected = None
        self.draw()

    def ask_promotion(self):
        # Simple dialog to choose promotion piece
        while True:
            s = simpledialog.askstring("Promotion", "Promote to (Q/R/B/N)?", initialvalue='Q')
            if s is None:
                return 'Q' # default
            s = s.strip().upper()
            if s in ['Q','R','B','N']:
                return s

    def push_history(self, move: Move, piece_before):
        files = 'abcdefgh'
        def sqname(rc):
            r, c = rc
            return f"{files[c]}{8-r}"
        name = piece_before[1] if piece_before[1] != 'P' else ''
        capture = self.board[move.dst[0]][move.dst[1]] is not None # after move, destination holds moved piece; capture implied earlier
        # We need capture detection from original board
        # Simpler: reconstruct capture by comparing pre-move board (piece at dst)
        # But we passed only piece_before; let's mark with 'x' if destination originally had opponent piece
        # For accuracy, we could accept minor ambiguity
        san = f"{name}{'x' if capture else ''}{sqname(move.dst)}"
        if move.promotion:
            san += f"={move.promotion}"
        # Check/checkmate marks after the move
        tmp = make_move(self.board, Move(move.src, move.dst, move.promotion)) # board here is pre-move; apply again on copy
        opp = WHITE if self.to_move == BLACK else BLACK
        mvs = legal_moves(tmp, opp)
        if not mvs:
            if in_check(tmp, opp):
                san += '#'
            else:
                san += ' (stalemate)'
        elif in_check(tmp, opp):
            san += '+'
        ply = self.history.size()+1
        prefix = f"{(ply+1)//2}. " if ply % 2 == 1 else ""
        self.history.insert(tk.END, prefix + san)


if __name__ == '__main__':
    app = ChessApp()
    app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\SNEHAL\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2074, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\SNEHAL\AppData\Local\Temp\ipykernel_11120\629960089.py", line 296, in on_click
    self.push_history(m, piece)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^
  File "C:\Users\SNEHAL\AppData\Local\Temp\ipykernel_11120\629960089.py", line 334, in push_history
    tmp = make_move(self.board, Move(move.src, move.dst, move.promotion)) # board here is pre-move; apply again on copy
  File "C:\Users\SNEHAL\AppData\Local\Temp\ipykernel_11120\629960089.py", line 170, in make_move
    if piece[1] == 'P' and (r2 == 0 or r2 == 7) and move.promotion:
       ~~~~~^^^
TypeError: 'NoneType' object is not subscriptable
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\SNEHAL\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", 